In [147]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
import math
import csv
import copy
import os

In [148]:
LiDAR_data = './raw_data/2022-12-10-16-53-36_Velodyne-VLP-16-Data.csv'

os.path.exists(LiDAR_data)

True

In [149]:
with open(LiDAR_data, newline='') as f:
    rows = list(csv.reader(f, delimiter=',', quotechar='"'))
    sph_lidar = np.zeros(shape=(len(rows) - 1, 3))
    headers = rows.pop(0)
    for index, row in enumerate(rows[1:]):
        # print(index, row[0])
        sph_lidar[index] = row[7:10]

In [150]:
indexer = {
    0:15,
    2:14,
    4:13,
    6:12,
    8:11,
    10:10,
    12:9,
    14:8,
    1:7,
    3:6,
    5:5,
    7:4,
    9:3,
    11:2,
    13:1,
    15:0,
}

sph_lidar[:,0] = np.array(list(map(indexer.get, sph_lidar[:,0])))

In [151]:
target_shape = tuple(sph_lidar[:,:2].astype(int).max(axis=0)[:2] + 1)
sph_lidar_frame = np.zeros(target_shape)
sph_lidar_frame[sph_lidar[:,0].astype(int),sph_lidar[:,1].astype(int)]=sph_lidar[:,2]

In [152]:
angle_mapper = {
    15:-15,
    14:-13,
    13:-11,
    12:-9,
    11:-7,
    10:-5,
    9:-3,
    8:-1,
    7:1,
    6:3,
    5:5,
    4:7,
    3:9,
    2:11,
    1:13,
    0:15,
}

In [153]:
def sph2cart(sph_frame):
    pts_lidar = np.zeros(shape=(sph_frame.size, 3))
    for rows in range(0,sph_frame.shape[0],1):
        for cols in range(0,sph_frame.shape[1],1):

            r = sph_frame[rows,cols]
            theta = np.array(list(map(angle_mapper.get, rows)))
            phi = cols/100

            pts_lidar[rows*sph_frame.shape[1]+cols,0] = r * math.sin(theta) * math.cos(phi)
            pts_lidar[rows*sph_frame.shape[1]+cols,1] =  r * math.sin(theta) * math.sin(phi)
            pts_lidar[rows*sph_frame.shape[1]+cols,2] = r * math.cos(theta)
    return pts_lidar

In [156]:
pts_lidar = np.zeros(shape=(sph_lidar_frame.size, 3))
print(pts_lidar.shape)

(576000, 3)


In [155]:
coords = o3d.geometry.TriangleMesh.create_coordinate_frame()
pcd_lidar = o3d.geometry.PointCloud()
pcd_lidar.points = o3d.utility.Vector3dVector(pts_lidar)

pcd_lidar_r = copy.deepcopy(pcd_lidar).translate((0, 0, 0))
pcd_lidar_r.rotate(pcd_lidar_r.get_rotation_matrix_from_xyz((-np.pi / 2, 0, 0)), center=(0, 0, 0))

pts_lidar_r = np.asarray(pcd_lidar_r.points)
o3d.visualization.draw_geometries([pcd_lidar_r])